In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import math

In [2]:
# Util for showing omages
def show(img, size=75):
    plt.figure(figsize = (size,size))
    
    if len(img.shape) == 2: # if grayscale, set cmap
        plt.imshow(img, cmap="gray", vmin=0, vmax=255)
    else: # if 3 channels, change from BGR to RGB
        plt.imshow(img[...,::-1])
        
    plt.show()

# Block Matching

In [7]:
from joblib import Parallel, delayed


def reject_outliers(data, m=2):
  outliers = np.where(abs(data - np.mean(data)) > m * np.std(data))
  data[outliers] = 0
  return data

def SAD(matA, matB):
  return np.sum(np.abs(matA - matB))

def SSD(matA, matB):
  return np.sum(np.square(matA - matB))

def get_row_matches(row, imgL, imgR, window, matching_fn):
  _, width, _ = imgL.shape
  # Window offset
  offset = int(window/2)
  result = []
  matched_before = dict()
  for colLeft in range(offset, width-offset):
    template = imgL[row-offset:row+offset +
                    1, colLeft-offset:colLeft+offset+1]
      

    bestMatchCol = -1
    bestMatchVal = 10000000

    for colRight in range(max(offset, colLeft-50), min(width-offset, colLeft+50)):
      if matched_before.get(colRight, 0) > 10:
        continue
      
      match = imgR[row-offset:row+offset+1,
                   colRight-offset:colRight+offset+1]
      matchVal = matching_fn(template, match) + np.abs(colRight - colLeft)

      if matchVal < bestMatchVal:
        bestMatchCol = colRight
        bestMatchVal = matchVal

    
    # Append disparity
    matched_before[bestMatchCol] = matched_before.get(bestMatchCol, 0) + 1
    result.append(np.abs(colLeft - bestMatchCol))
  return result


def get_block_matches(imgL, imgR, window, matching_fn):
  height, _, _ = imgL.shape
  offset = int(window/2)

  imgL = cv.copyMakeBorder(imgL, offset, offset,  offset,  offset, cv.BORDER_CONSTANT, value=[0])
  imgR = cv.copyMakeBorder(imgR, offset, offset,  offset,
                         offset, cv.BORDER_CONSTANT, value=[0])

  results = Parallel(n_jobs=10)(delayed(get_row_matches)(i, imgL, imgR, window,
                                              matching_fn) for i in range(offset, height-offset))

  # results = []
  # for row in range(offset, height-offset):
  #   results.append(get_row_matches(row, imgL, imgR, window, matching_fn))
  #   print(f"{row}/{height}")
  
  results = np.array(results)
  results = reject_outliers(results)
  results = results * (255.0/results.max())
  results = results.astype(np.uint8)
  return results

In [8]:
for image_no in [1, 2, 3]:
  # imgL = cv.cvtColor(cv.imread(f"./l{image_no}.png"), cv.COLOR_BGR2GRAY)
  # imgR = cv.cvtColor(cv.imread(f"./r{image_no}.png"), cv.COLOR_BGR2GRAY)
  imgL = cv.imread(f"./l{image_no}.png")
  imgR = cv.imread(f"./r{image_no}.png")
  
  for window in [3, 5, 9]:
    print("Window size: ", window)

    print("Calculating SAD ... ")
    disparity_map = get_block_matches(imgL, imgR, window, SAD)
    print(disparity_map)
    cv.imwrite(f"./block_matching/output_{image_no}_w{window}_SAD.png", disparity_map)


    print("Calculating SSD ... ")
    disparity_map = get_block_matches(imgL, imgR, window, SSD)
    print(disparity_map)
    cv.imwrite(
        f"./block_matching/output_{image_no}_w{window}_SSD.png", disparity_map)

  

Window size:  3
Calculating SAD ... 
[[  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 ...
 [  0   6  12 ...   0   0   0]
 [  0   6  12 ... 153   6   0]
 [  0   6  12 ...   0   0   0]]
Calculating SSD ... 
[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0 42 ... 42  0  0]
 [ 0  0 42 ... 42  0  0]
 [ 0  0 42 ... 56 56  0]]
Window size:  5
Calculating SAD ... 
[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0 11 ... 11  5  0]
 [ 0  0 11 ... 11  5  0]
 [ 0  0 11 ...  0  5  0]]
Calculating SSD ... 
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Window size:  9
Calculating SAD ... 
[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ... 11  0  0]
 ...
 [ 0  5 11 ... 11  5  0]
 [ 0  5 11 ... 11  0  0]
 [ 0  5 11 ...  5  0  5]]
Calculating SSD ... 
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 

In [3]:
def plot_matches(imgL, imgR, distances, c, dp, row):
    height, width, rest_dims = imgL.shape[0], imgL.shape[1], imgL.shape[2:]

    output = np.zeros((height+1, width+1, *rest_dims))
    # left image row
    output[0, 1:, ...] = imgL[row]
    # right image row
    output[1:, 0, ...] = imgR[row]

    i, j = height - 1, width - 1

    while i > 0 or j > 0:
        if i == 0:
            output = cv.line(output, (j+1, i+1), (j, i+1), (255, 255, 255), 1)
            j -= 1
        elif j == 0:
            output = cv.line(output, (j+1, i+1), (j+1, i), (255, 255, 255), 1)
            i -= 1
        elif dp[i][j] == dp[i-1][j] + c:
            output = cv.line(output, (j+1, i+1), (j+1, i), (255, 255, 255), 1)
            i -= 1
        elif dp[i][j] == dp[i][j-1] + c:
            output = cv.line(output, (j+1, i+1), (j, i+1), (255, 255, 255), 1)
            j -= 1
        else:
            output = cv.line(output, (j+1, i+1), (j, i), (255, 255, 255), 1)
            i -= 1
            j -= 1
    return output